In [1]:
import os
import json
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
import glob


In [2]:
BASE_PATH = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation"
CONF_MATRIX_DIR = os.path.join(BASE_PATH, "FINAL_DATEN/confusion_matrices_hyper_small")
EVAL_RESULTS_SAVE_PATH = os.path.join(BASE_PATH, "FINAL_DATEN", "evaluation_results_map_small.json")

# Debug-Ausgabe der gesetzten Pfade:
print("BASE_PATH:", BASE_PATH)
print("CONF_MATRIX_DIR:", CONF_MATRIX_DIR)
print("Evaluation Results Save Path:", EVAL_RESULTS_SAVE_PATH)


BASE_PATH: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation
CONF_MATRIX_DIR: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_hyper_small
Evaluation Results Save Path: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/evaluation_results_map_small.json


In [3]:
def compute_confusion_matrix_metrics(cm):
    """
    Berechnet mIoU und FWIoU für eine gegebene Confusion Matrix.
    Rückgabe: Dictionary mit mIoU und FWIoU.
    """
    # Intersection, Ground Truth und Predicted Summe
    intersection = torch.diag(cm)
    ground_truth_set = cm.sum(1)
    predicted_set = cm.sum(0)
    
    # Berechne die IoU für jede Klasse
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / (union + 1e-6)
    mIoU = torch.mean(IoU).item()
    
    # Berechnung des Frequency Weighted IoU (FWIoU)
    total_pixels_all = cm.sum()
    FWIoU = (ground_truth_set * IoU).sum() / (total_pixels_all + 1e-6)
    FWIoU = FWIoU.item()
    
    return {
        "mIoU": mIoU,
        "FWIoU": FWIoU
    }

# Suche nach allen Confusion Matrix-Dateien im definierten Ordner
conf_files = glob.glob(os.path.join(CONF_MATRIX_DIR, "*_confusion_matrix.pt"))
metrics_summary = {}

for file_path in conf_files:
    model_name = os.path.basename(file_path).split("_confusion_matrix.pt")[0]
    cm = torch.load(file_path)
    metrics = compute_confusion_matrix_metrics(cm)
    metrics_summary[model_name] = metrics
    print(f"Model: {model_name}, mIoU: {metrics['mIoU']:.4f}, FWIoU: {metrics['FWIoU']:.4f}")

# Speichern der Metriken als JSON-Datei
with open(EVAL_RESULTS_SAVE_PATH, "w", encoding="utf-8") as f:
    json.dump(metrics_summary, f, indent=4)

print(f"Evaluation metrics saved to: {EVAL_RESULTS_SAVE_PATH}")


Model: fcn_resnet101, mIoU: 0.1541, FWIoU: 0.8020
Model: deeplabv3_resnet101, mIoU: 0.1527, FWIoU: 0.8085
Model: deeplabv3_resnet50, mIoU: 0.1465, FWIoU: 0.8030
Model: fcn_resnet50, mIoU: 0.1510, FWIoU: 0.7959
Evaluation metrics saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/evaluation_results_map_small.json


In [4]:
def compute_confusion_matrix_metrics_ignore_empty(cm):
    """
    Berechnet mIoU und FWIoU für eine gegebene Confusion Matrix,
    wobei Klassen, die im Ground Truth nicht vorkommen (leere Klassen),
    ignoriert werden.
    
    Rückgabe: Dictionary mit mIoU und FWIoU.
    """
    # Berechne Intersection, Ground Truth- und Predicted-Summen (als float)
    intersection = torch.diag(cm)
    ground_truth_set = cm.sum(1).float()
    predicted_set = cm.sum(0).float()
    
    # Berechne die Union und per-Klassen IoU
    union = ground_truth_set + predicted_set - intersection.float()
    IoU = intersection.float() / (union + 1e-6)
    
    # Maske für Klassen, die im Ground Truth vorhanden sind (also nicht leer)
    valid_mask = ground_truth_set > 0

    # Berechne mIoU nur für die validen Klassen
    if valid_mask.sum() > 0:
        mIoU = torch.mean(IoU[valid_mask]).item()
    else:
        mIoU = 0.0

    # Berechne FWIoU basierend auf den validen Klassen
    total_valid_pixels = ground_truth_set[valid_mask].sum() + 1e-6
    FWIoU = (ground_truth_set[valid_mask] * IoU[valid_mask]).sum() / total_valid_pixels
    FWIoU = FWIoU.item()
    
    return {"mIoU": mIoU, "FWIoU": FWIoU}

# Suche nach allen Confusion Matrix-Dateien im definierten Ordner
conf_files = glob.glob(os.path.join(CONF_MATRIX_DIR, "*_confusion_matrix.pt"))
metrics_summary_corrected = {}

for file_path in conf_files:
    model_name = os.path.basename(file_path).split("_confusion_matrix.pt")[0]
    cm = torch.load(file_path)
    metrics = compute_confusion_matrix_metrics_ignore_empty(cm)
    metrics_summary_corrected[model_name] = metrics
    print(f"Model: {model_name}, Corrected Metrics: mIoU = {metrics['mIoU']:.4f}, FWIoU = {metrics['FWIoU']:.4f}")

# Erstelle den Pfad für die korrigierte JSON-Datei, basierend auf EVAL_RESULTS_SAVE_PATH
corrected_path = EVAL_RESULTS_SAVE_PATH.replace(".json", "_corrected.json")
with open(corrected_path, "w", encoding="utf-8") as f:
    json.dump(metrics_summary_corrected, f, indent=4)

print(f"Corrected evaluation metrics saved to: {corrected_path}")


Model: fcn_resnet101, Corrected Metrics: mIoU = 0.2895, FWIoU = 0.8020
Model: deeplabv3_resnet101, Corrected Metrics: mIoU = 0.2868, FWIoU = 0.8085
Model: deeplabv3_resnet50, Corrected Metrics: mIoU = 0.2752, FWIoU = 0.8030
Model: fcn_resnet50, Corrected Metrics: mIoU = 0.2837, FWIoU = 0.7959
Corrected evaluation metrics saved to: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/evaluation_results_map_small_corrected.json


In [5]:
# %% Cell 6: Display IoU per Class with Class Names, Frequency and Save to JSON

# Parameter: Ausgabe-Pfad für die JSON-Datei
OUTPUT_JSON_PATH = os.path.join(BASE_PATH, "FINAL_DATEN", "iou_per_class_results_map_small.json")

# Lade die Konfigurationsdatei, um die Klassennamen zu erhalten
config_path = os.path.join(BASE_PATH, "Mapillary_Vistas", "config_v2.0.json")
with open(config_path, "r") as f:
    config = json.load(f)

# Erstelle ein Dictionary, das jedem Index den lesbaren Namen zuordnet
class_names = {i: info["readable"] for i, info in enumerate(config["labels"])}

def compute_iou_and_frequency(cm):
    """
    Berechnet die IoU pro Klasse sowie die relative Häufigkeit (Frequency) 
    basierend auf der Anzahl der Ground Truth-Pixel (in Prozent).
    
    Rückgabe:
        iou_per_class (Tensor): IoU pro Klasse.
        frequency (Tensor): Frequenz jeder Klasse in Prozent.
    """
    epsilon = 1e-6
    intersection = torch.diag(cm).float()
    row_sum = cm.sum(1).float()   # Ground Truth-Pixel pro Klasse
    col_sum = cm.sum(0).float()   # Vorhergesagte Pixel pro Klasse
    union = row_sum + col_sum - intersection + epsilon
    iou_per_class = intersection / union
    
    total_pixels = cm.sum().float() + epsilon
    frequency = (row_sum / total_pixels) * 100.0  # in Prozent
    return iou_per_class, frequency

# Suche nach allen Confusion Matrix-Dateien im definierten Ordner
conf_files = glob.glob(os.path.join(CONF_MATRIX_DIR, "*_confusion_matrix.pt"))

results_dict = {}
NUM_CLASSES = cm.shape[0]


# Ausgabe pro Modell
for file_path in conf_files:
    model_name = os.path.basename(file_path).split("_confusion_matrix.pt")[0]
    cm = torch.load(file_path)
    iou_values, freq_values = compute_iou_and_frequency(cm)
    
    # Erstelle eine Liste mit per-Klassen Metriken
    per_class_metrics = []
    for i in range(NUM_CLASSES):
        name = class_names.get(i, f"Class {i}")
        per_class_metrics.append({
            "class_index": i,
            "class_name": name,
            "IoU": round(iou_values[i].item(), 4),
            "frequency_percent": round(freq_values[i].item(), 2)
        })
    
    # Speichere zusätzlich Mean IoU und FWIoU (über compute_confusion_matrix_metrics oder compute_confusion_matrix_metrics_ignore_empty)
    metrics = compute_confusion_matrix_metrics(cm)
    
    results_dict[model_name] = {
        "mIoU": round(metrics["mIoU"], 4),
        "FWIoU": round(metrics["FWIoU"], 4),
        "per_class": per_class_metrics
    }
    
    # Optionale Konsolenausgabe
    print("=" * 70)
    print(f"Model: {model_name}")
    print(f"Mean IoU: {metrics['mIoU']:.4f}")
    print(f"FWIoU: {metrics['FWIoU']:.4f}")
    print("\nPer-Klassen Metriken:")
    print("{:<12} {:<40} {:>10} {:>15}".format("Class Index", "Class Name", "IoU", "Frequency (%)"))
    print("-" * 70)
    for entry in per_class_metrics:
        print("{:<12} {:<40} {:>10.4f} {:>15.2f}".format(
            entry["class_index"], entry["class_name"], entry["IoU"], entry["frequency_percent"]
        ))
    print("\n")

# Speichern der Ergebnisse als JSON
OUTPUT_JSON_PATH = os.path.join(BASE_PATH, "FINAL_DATEN", "iou_per_class_results_map_small.json")
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(results_dict, f, indent=4)
print(f"Ergebnisse wurden in JSON gespeichert: {OUTPUT_JSON_PATH}")


Model: fcn_resnet101
Mean IoU: 0.1541
FWIoU: 0.8020

Per-Klassen Metriken:
Class Index  Class Name                                      IoU   Frequency (%)
----------------------------------------------------------------------
0            Bird                                         0.0000            0.00
1            Ground Animal                                0.0000            0.00
2            Ambiguous Barrier                            0.0000            0.00
3            Concrete Block                               0.5939            0.31
4            Curb                                         0.4815            0.85
5            Fence                                        0.4607            1.27
6            Guard Rail                                   0.5903            0.24
7            Barrier                                      0.0000            0.00
8            Road Median                                  0.0000            0.00
9            Road Side                      